In [1]:
import numpy as np
import pickle
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image, ImageFilter
import autoencoder
import display_layer
import gradient
import sample_patches
from importlib import reload
from scipy.optimize import minimize
import time

In [2]:
def norm_weight(x):
    return (x - np.min(x)) / (np.max(x) - np.min(x))

In [3]:
reload(display_layer)
reload(sample_patches)
reload(autoencoder)
reload(gradient)

<module 'gradient' from '/home/anuar/Рабочий стол/Student/3 kurs/2 sm/Prack/task 5/gradient.py'>

##Первый пункт.

<font size=4, face="Times New Roman"> Генерируются случайные участки изображения 8x8. Находится в модуле *sample_pathces.py*.</font>

##Второй пункт.

<font size=4, face="Times New Roman"> Эти участки изображения нормализуются по правилу трех сигма. </font>

##Третий пункт.

<font size=4, face="Times New Roman"> В модуле *autoencoder.py* реализован метод обратного распространения ошибки с учетом использования регуляризатора разреженности. </font>

##Четвертый пункт.

<font size=4, face="Times New Roman"> Функция потерь многослойного разреженного автокодировщика реализован в модуле *autoencoder.py*. При написании кода максимально были использованы матричные вычисления. </font>

##Пятый пункт.

###A). Проверяется на отдельной простой функции.

In [5]:
gradient.check_gradient()

Difference between gradients:  1.54679262192e-11


<font size=4, face="Times New Roman"> Это означает, что функция <b> compute_gradient </b> реализована правильно. </font>

###B). Проверяется на функции потерь автокодировщика.

<font size=4, face="Times New Roman"> Необходимо было проверить корректность вычислений градиентов в методе обратного распространения ошибки с помощью функции <b> compute_gradient </b>. </font>

In [18]:
#проверяем на нейронной сети с одним слоем
hidden_size = np.array([7, 4, 7])
visible_size = 8
theta = autoencoder.initialize(hidden_size, visible_size)
patches = np.random.rand(10, 8)

In [3]:
#сама функция потерь
def J(theta):
    return autoencoder.autoencoder_loss(theta, visible_size, hidden_size, 
                                        lambda_, sparsity_param, beta, patches)

In [20]:
grad_1 = J(theta)[1]
grad_2 = gradient.compute_gradient(J, theta)

In [21]:
np.linalg.norm(grad_1 - grad_2)

1.3237701933209175e-09

<font size=4, face="Times New Roman"> Как мы видим ошибка составляет <b> $ 1.3 * 10^{-9} $ </b>, что говорит о том, что градиенты вычислены правильно. </font>

##Шестой пункт.

<font size=4, face="Times New Roman"> Минимизация проводится с помощью функции <b> minimize. </b> </font>

In [10]:
%time minimize(J, theta, method='L-BFGS-B', jac=True)

CPU times: user 73.2 ms, sys: 0 ns, total: 73.2 ms
Wall time: 72.6 ms


 message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     jac: array([  6.93753490e-07,   7.21060756e-07,  -1.16941775e-06,
         7.19558136e-07,   1.96337209e-06,   2.08306887e-06,
        -2.91996726e-07,   1.04291861e-06,  -1.69551315e-06,
        -2.06629925e-06,  -1.46221081e-06,  -2.78042087e-06,
        -1.39557868e-06,  -1.35817462e-06,  -9.34283373e-07,
        -8.04019283e-07,  -5.25174995e-07,  -7.19739862e-07,
        -2.42525282e-06,  -2.33098200e-06,   2.70410597e-06,
         7.48379633e-07,   6.28876872e-07,   2.19052432e-06,
        -1.18237799e-07,   2.23575735e-06,   3.43707996e-06,
         4.44717884e-06,  -1.67879664e-06,   1.73494609e-07,
        -2.73060135e-06,   5.66763554e-07,  -4.37128967e-07,
        -3.62015336e-06,  -4.48749697e-06,  -5.10747384e-07,
        -1.48451888e-06,  -2.99467333e-06,  -7.90450542e-07,
         2.01072492e-06,  -1.09779442e-06,   1.12659354e-06,
        -1.53959292e-06,  -8.38347861e-07,   7.91576976e-07,
         5.39

##Седьмой пункт.

<font size=4, face='Times New Roman'> Необходимо "прогнать" патчи через автокодировщик и вывести веса на скрытых слоях </font>

In [4]:
with open('unlabeled.pk', 'rb') as f:
    unl = pickle.load(f)

In [5]:
hidden_size = np.array([75])
visible_size = 192
theta = autoencoder.initialize(hidden_size, visible_size)
%time patches = sample_patches.sample_patches(unl['X'])
beta = 3
sparsity_param = 1e-2
lambda_ = 1e-5

CPU times: user 16.9 s, sys: 0 ns, total: 16.9 s
Wall time: 1min 2s


In [243]:
%time display_layer.display_layer(patches[:256], filename='patches_unl.png')

CPU times: user 24.5 ms, sys: 0 ns, total: 24.5 ms
Wall time: 24 ms


<font size=4, face='Times New Roman'> Время выполнения одной итерации <b> autoencoder_loss </b> составляет 512 ms. </font>

In [220]:
%time autoencoder.autoencoder_loss(theta, visible_size, hidden_size, lambda_, sparsity_param, beta, patches)

CPU times: user 500 ms, sys: 12.2 ms, total: 513 ms
Wall time: 512 ms


(168.64542800892752,
 array([ 1.08329654,  1.08123896,  1.08203725, ...,  0.02288452,
        -0.03893653,  0.01575775]))

In [8]:
%time result = minimize(J, theta, method='L-BFGS-B', jac=True, tol=1e-5, options={'max_iter': 4000})

CPU times: user 9min 57s, sys: 11.5 s, total: 10min 9s
Wall time: 10min 10s


/root/anaconda3/lib/python3.4/site-packages/IPython/kernel/__main__.py:1: OptimizeWarning: Unknown solver options: max_iter
  if __name__ == '__main__':


<font size=4, face="Times New Roman"> Веса между первым и вторым слоями сети. </font>

In [253]:
layer_number = 1
%time activation, W = autoencoder.autoencoder_transform(result['x'], visible_size, hidden_size, layer_number, patches)
display_layer.display_layer(norm_weight(W), filename='layer1_W_1.bmp')

CPU times: user 167 ms, sys: 179 µs, total: 168 ms
Wall time: 468 ms


<font size=4, face="Times New Roman"> Веса между вторым и третьем слоями сети. </font>

In [254]:
layer_number = 2
%time activation, W = autoencoder.autoencoder_transform(result['x'], visible_size, hidden_size, layer_number, patches)
display_layer.display_layer(norm_weight(W.T), filename='layer2_W_1.bmp')

CPU times: user 181 ms, sys: 36 µs, total: 181 ms
Wall time: 180 ms


<font size=4.5, face="Times New Roman"> Изменение гиперпараметров сети: </font>

$\lambda = 10^{-3}$

In [68]:
%time patches = sample_patches.sample_patches(train['X'])

CPU times: user 14.5 s, sys: 32.9 ms, total: 14.5 s
Wall time: 17.6 s


In [81]:
hidden_size = np.array([75])
visible_size = 192
theta = autoencoder.initialize(hidden_size, visible_size)
beta = 3
sparsity_param = 1e-2
lambda_ = 1e-3

In [82]:
%time result = minimize(J, theta, method='L-BFGS-B', jac=True, tol=1e-5, options={'max_iter': 4000})

CPU times: user 31.4 s, sys: 0 ns, total: 31.4 s
Wall time: 31.4 s


/root/anaconda3/lib/python3.4/site-packages/IPython/kernel/__main__.py:1: OptimizeWarning: Unknown solver options: max_iter
  if __name__ == '__main__':


In [83]:
layer_number = 1
%time activation, W = autoencoder.autoencoder_transform(result['x'], visible_size, hidden_size, layer_number, patches)
display_layer.display_layer(norm_weight(W), filename='layer1_W_1_lambda.png')

CPU times: user 191 ms, sys: 0 ns, total: 191 ms
Wall time: 191 ms


In [84]:
layer_number = 2
%time activation, W = autoencoder.autoencoder_transform(result['x'], visible_size, hidden_size, layer_number, patches)
display_layer.display_layer(norm_weight(W.T), filename='layer2_W_1_lambda.png')

CPU times: user 175 ms, sys: 0 ns, total: 175 ms
Wall time: 173 ms


$\rho = 0.1$

In [73]:
hidden_size = np.array([75])
visible_size = 192
theta = autoencoder.initialize(hidden_size, visible_size)
beta = 3
sparsity_param = 0.1
lambda_ = 1e-5

In [74]:
%time result = minimize(J, theta, method='L-BFGS-B', jac=True, tol=1e-5, options={'max_iter': 4000})

CPU times: user 7min 15s, sys: 120 ms, total: 7min 15s
Wall time: 7min 15s


/root/anaconda3/lib/python3.4/site-packages/IPython/kernel/__main__.py:1: OptimizeWarning: Unknown solver options: max_iter
  if __name__ == '__main__':


In [79]:
layer_number = 1
%time activation, W = autoencoder.autoencoder_transform(result['x'], visible_size, hidden_size, layer_number, patches)
display_layer.display_layer(norm_weight(W), filename='layer1_W_1_rho.png')

CPU times: user 174 ms, sys: 213 µs, total: 174 ms
Wall time: 173 ms


In [80]:
layer_number = 2
%time activation, W = autoencoder.autoencoder_transform(result['x'], visible_size, hidden_size, layer_number, patches)
display_layer.display_layer(norm_weight(W.T), filename='layer2_W_1_rho.png')

CPU times: user 171 ms, sys: 3.86 ms, total: 175 ms
Wall time: 174 ms


##Восьмой пункт.

<font size=4, face="Times New Roman"> Обучаем нейронную сеть на *unlabeled.pk* и дальше применяем классификаторы RandomForest и LogisticRegression. </font>

In [9]:
with open('train.pk', 'rb') as f:
    train = pickle.load(f)

In [10]:
with open('test.pk', 'rb') as f:
    test = pickle.load(f)

In [4]:
def image_patches(image, step, patch_size=8):
    N, D = image.shape
    d = int(np.sqrt(D / 3))
    
    n = int((d - patch_size) / step) + 1
    patches = np.zeros((N, n, n, patch_size * patch_size * 3))
    img = image.reshape((N, d, d, 3))
    for i in range(n):
        for j in range(n):
            patch = img[: , i * step: i * step + patch_size, j * step: j * step + patch_size, :]     
            patches[: , i, j, : ] = patch.reshape((N, patch_size * patch_size * 3))
    return sample_patches.normalize_data(patches.reshape((N * n * n, patch_size * patch_size * 3)))

<font size=3, face="Times New Roman"> Примечание: время работы для выделения патчей оказалось долгим из-за того, что все патчи были не в основной памяти, а хранились в swap'e </font>

<font size=5, face="Times New Roman"> step = 32 </font>

In [101]:
step = 32
%time patches = image_patches(train['X'], step)

CPU times: user 280 ms, sys: 0 ns, total: 280 ms
Wall time: 279 ms


In [102]:
layer_number = 2
%time activation = autoencoder.autoencoder_transform(result['x'], visible_size, hidden_size, layer_number, patches)[0]

CPU times: user 764 ms, sys: 0 ns, total: 764 ms
Wall time: 763 ms


In [103]:
N, D = activation.shape
n_samples = train['X'].shape[0]
activation = activation.reshape((n_samples, N * D // n_samples))

In [5]:
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.linear_model import LogisticRegression as LR
from sklearn.metrics import accuracy_score as accuracy

In [105]:
alg_RFC_1 = RFC(n_estimators=20)
alg_LR_1 = LR()
alg_RFC_2 = RFC(n_estimators=20)
alg_LR_2 = LR()

In [106]:
#на признаках, выделенных автокодировщиком
acc_RFC_1 = []
acc_LR_1 = []

#на исходных признаках (интенсивность каналов)
acc_RFC_2 = []
acc_LR_2 = []

In [107]:
%time alg_RFC_1.fit(activation, train['y'].reshape(train['y'].shape[0]))

CPU times: user 2.26 s, sys: 0 ns, total: 2.26 s
Wall time: 2.8 s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [108]:
%time alg_RFC_2.fit(train['X'], train['y'].reshape(train['y'].shape[0]))

CPU times: user 12.5 s, sys: 0 ns, total: 12.5 s
Wall time: 12.5 s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [109]:
%time alg_LR_1.fit(activation, train['y'].reshape(train['y'].shape[0]))

CPU times: user 4.03 s, sys: 7.64 ms, total: 4.04 s
Wall time: 4.35 s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)

In [110]:
%time alg_LR_2.fit(train['X'], train['y'].reshape(train['y'].shape[0]))

CPU times: user 23min 36s, sys: 1.6 s, total: 23min 38s
Wall time: 27min 35s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)

In [111]:
%time patches = image_patches(test['X'], step)

CPU times: user 461 ms, sys: 169 ms, total: 630 ms
Wall time: 21.3 s


In [112]:
layer_number = 2
%time activation = autoencoder.autoencoder_transform(result['x'], visible_size, hidden_size, layer_number, patches)[0]

CPU times: user 1.27 s, sys: 20 ms, total: 1.29 s
Wall time: 2.71 s


In [113]:
N, D = activation.shape
n_samples = test['X'].shape[0]
activation = activation.reshape((n_samples, N * D // n_samples))

In [114]:
%time y_predict_RFC_1 = alg_RFC_1.predict(activation)

CPU times: user 81 ms, sys: 20 ms, total: 101 ms
Wall time: 2.45 s


In [115]:
%time y_predict_RFC_2 = alg_RFC_2.predict(test['X'])

CPU times: user 455 ms, sys: 91.9 ms, total: 547 ms
Wall time: 13.5 s


In [116]:
%time y_predict_LR_1 = alg_LR_1.predict(activation)

CPU times: user 28.6 ms, sys: 0 ns, total: 28.6 ms
Wall time: 1.69 s


In [117]:
%time y_predict_LR_2 = alg_LR_2.predict(test['X'])

CPU times: user 1.56 s, sys: 1.04 s, total: 2.59 s
Wall time: 2min 22s


In [118]:
acc_RFC_1.append(accuracy(test['y'].reshape(test['y'].shape[0]), y_predict_RFC_1))
acc_LR_1.append(accuracy(test['y'].reshape(test['y'].shape[0]), y_predict_LR_1))
acc_RFC_2.append(accuracy(test['y'].reshape(test['y'].shape[0]), y_predict_RFC_2))
acc_LR_2.append(accuracy(test['y'].reshape(test['y'].shape[0]), y_predict_LR_2))

<font size=5, face="Times New Roman"> step = 28 </font>

In [119]:
step = 28
%time patches = image_patches(train['X'], step)

CPU times: user 490 ms, sys: 425 ms, total: 915 ms
Wall time: 1min 6s


In [120]:
layer_number = 2
%time activation = autoencoder.autoencoder_transform(result['x'], visible_size, hidden_size, layer_number, patches)[0]

CPU times: user 1.49 s, sys: 121 ms, total: 1.62 s
Wall time: 21 s


In [121]:
N, D = activation.shape
n_samples = train['X'].shape[0]
activation = activation.reshape((n_samples, N * D // n_samples))

In [122]:
%time alg_RFC_1.fit(activation, train['y'].reshape(train['y'].shape[0]))

CPU times: user 3.03 s, sys: 28.2 ms, total: 3.06 s
Wall time: 14.2 s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [123]:
%time alg_LR_1.fit(activation, train['y'].reshape(train['y'].shape[0]))

CPU times: user 6.56 s, sys: 28.1 ms, total: 6.59 s
Wall time: 12.9 s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)

In [124]:
%time patches = image_patches(test['X'], step)

CPU times: user 760 ms, sys: 405 ms, total: 1.17 s
Wall time: 1min 42s


In [125]:
%time activation = autoencoder.autoencoder_transform(result['x'], visible_size, hidden_size, layer_number, patches)[0]

CPU times: user 2.24 s, sys: 92.3 ms, total: 2.33 s
Wall time: 17.9 s


In [126]:
N, D = activation.shape
n_samples = test['X'].shape[0]
activation = activation.reshape((n_samples, N * D // n_samples))

In [127]:
%time y_predict_RFC_1 = alg_RFC_1.predict(activation)

CPU times: user 106 ms, sys: 187 µs, total: 106 ms
Wall time: 3.94 s


In [128]:
%time y_predict_LR_1 = alg_LR_1.predict(activation)

CPU times: user 44.7 ms, sys: 0 ns, total: 44.7 ms
Wall time: 605 ms


In [129]:
acc_RFC_1.append(accuracy(test['y'].reshape(test['y'].shape[0]), y_predict_RFC_1))
acc_LR_1.append(accuracy(test['y'].reshape(test['y'].shape[0]), y_predict_LR_1))

<font size=5, face="Times New Roman"> step = 24 </font>

In [130]:
step = 24
%time patches = image_patches(train['X'], step)

CPU times: user 479 ms, sys: 36.2 ms, total: 516 ms
Wall time: 24 s


In [131]:
layer_number = 2
%time activation = autoencoder.autoencoder_transform(result['x'], visible_size, hidden_size, layer_number, patches)[0]

CPU times: user 1.4 s, sys: 24.3 ms, total: 1.43 s
Wall time: 10.5 s


In [132]:
N, D = activation.shape
n_samples = train['X'].shape[0]
activation = activation.reshape((n_samples, N * D // n_samples))

In [133]:
%time alg_RFC_1.fit(activation, train['y'].reshape(train['y'].shape[0]))

CPU times: user 2.99 s, sys: 4.01 ms, total: 2.99 s
Wall time: 4.19 s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [134]:
%time alg_LR_1.fit(activation, train['y'].reshape(train['y'].shape[0]))

CPU times: user 6.41 s, sys: 0 ns, total: 6.41 s
Wall time: 7.01 s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)

In [135]:
%time patches = image_patches(test['X'], step)

CPU times: user 922 ms, sys: 88.8 ms, total: 1.01 s
Wall time: 58 s


In [136]:
%time activation = autoencoder.autoencoder_transform(result['x'], visible_size, hidden_size, layer_number, patches)[0]

CPU times: user 2.69 s, sys: 3.95 ms, total: 2.69 s
Wall time: 3.24 s


In [137]:
N, D = activation.shape
n_samples = test['X'].shape[0]
activation = activation.reshape((n_samples, N * D // n_samples))

In [138]:
%time y_predict_RFC_1 = alg_RFC_1.predict(activation)

CPU times: user 86.4 ms, sys: 3.99 ms, total: 90.3 ms
Wall time: 293 ms


In [139]:
%time y_predict_LR_1 = alg_LR_1.predict(activation)

CPU times: user 41.2 ms, sys: 0 ns, total: 41.2 ms
Wall time: 40.8 ms


In [140]:
acc_RFC_1.append(accuracy(test['y'].reshape(test['y'].shape[0]), y_predict_RFC_1))
acc_LR_1.append(accuracy(test['y'].reshape(test['y'].shape[0]), y_predict_LR_1))

<font size=5, face="Times New Roman"> step = 20 </font>

In [141]:
step = 20
%time patches = image_patches(train['X'], step)

CPU times: user 702 ms, sys: 132 ms, total: 834 ms
Wall time: 31 s


In [142]:
layer_number = 2
%time activation = autoencoder.autoencoder_transform(result['x'], visible_size, hidden_size, layer_number, patches)[0]

CPU times: user 2.66 s, sys: 180 ms, total: 2.84 s
Wall time: 30.8 s


In [143]:
N, D = activation.shape
n_samples = train['X'].shape[0]
activation = activation.reshape((n_samples, N * D // n_samples))

In [144]:
%time alg_RFC_1.fit(activation, train['y'].reshape(train['y'].shape[0]))

CPU times: user 4.16 s, sys: 0 ns, total: 4.16 s
Wall time: 4.63 s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [145]:
%time alg_LR_1.fit(activation, train['y'].reshape(train['y'].shape[0]))

CPU times: user 11.7 s, sys: 23.9 ms, total: 11.7 s
Wall time: 15.1 s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)

In [146]:
%time patches = image_patches(test['X'], step)

CPU times: user 1.28 s, sys: 317 ms, total: 1.6 s
Wall time: 42.9 s


In [147]:
%time activation = autoencoder.autoencoder_transform(result['x'], visible_size, hidden_size, layer_number, patches)[0]

CPU times: user 3.76 s, sys: 201 ms, total: 3.96 s
Wall time: 22.1 s


In [148]:
N, D = activation.shape
n_samples = test['X'].shape[0]
activation = activation.reshape((n_samples, N * D // n_samples))

In [149]:
%time y_predict_RFC_1 = alg_RFC_1.predict(activation)

CPU times: user 116 ms, sys: 0 ns, total: 116 ms
Wall time: 1.05 s


In [150]:
%time y_predict_LR_1 = alg_LR_1.predict(activation)

CPU times: user 65.7 ms, sys: 0 ns, total: 65.7 ms
Wall time: 416 ms


In [151]:
acc_RFC_1.append(accuracy(test['y'].reshape(test['y'].shape[0]), y_predict_RFC_1))
acc_LR_1.append(accuracy(test['y'].reshape(test['y'].shape[0]), y_predict_LR_1))

<font size=5, face="Times New Roman"> step = 16 </font>

In [152]:
step = 16
%time patches = image_patches(train['X'], step)

CPU times: user 1.03 s, sys: 24.5 ms, total: 1.05 s
Wall time: 10.2 s


In [153]:
layer_number = 2
%time activation = autoencoder.autoencoder_transform(result['x'], visible_size, hidden_size, layer_number, patches)[0]

CPU times: user 3.17 s, sys: 0 ns, total: 3.17 s
Wall time: 3.3 s


In [154]:
N, D = activation.shape
n_samples = train['X'].shape[0]
activation = activation.reshape((n_samples, N * D // n_samples))

In [155]:
%time alg_RFC_1.fit(activation, train['y'].reshape(train['y'].shape[0]))

CPU times: user 4.57 s, sys: 3.98 ms, total: 4.58 s
Wall time: 6.22 s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [156]:
%time alg_LR_1.fit(activation, train['y'].reshape(train['y'].shape[0]))

CPU times: user 16.3 s, sys: 3.84 ms, total: 16.3 s
Wall time: 17.5 s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)

In [157]:
%time patches = image_patches(test['X'], step)

CPU times: user 1.68 s, sys: 169 ms, total: 1.85 s
Wall time: 37.1 s


In [158]:
%time activation = autoencoder.autoencoder_transform(result['x'], visible_size, hidden_size, layer_number, patches)[0]

CPU times: user 5.48 s, sys: 929 ms, total: 6.41 s
Wall time: 3min 11s


In [159]:
N, D = activation.shape
n_samples = test['X'].shape[0]
activation = activation.reshape((n_samples, N * D // n_samples))

In [160]:
%time y_predict_RFC_1 = alg_RFC_1.predict(activation)

CPU times: user 198 ms, sys: 56.4 ms, total: 255 ms
Wall time: 21.3 s


In [161]:
%time y_predict_LR_1 = alg_LR_1.predict(activation)

CPU times: user 95.3 ms, sys: 0 ns, total: 95.3 ms
Wall time: 887 ms


In [162]:
acc_RFC_1.append(accuracy(test['y'].reshape(test['y'].shape[0]), y_predict_RFC_1))
acc_LR_1.append(accuracy(test['y'].reshape(test['y'].shape[0]), y_predict_LR_1))

<font size=5, face="Times New Roman"> step = 12 </font>

In [163]:
step = 12
%time patches = image_patches(train['X'], step)

CPU times: user 1.82 s, sys: 318 ms, total: 2.14 s
Wall time: 1min 9s


In [164]:
layer_number = 2
%time activation = autoencoder.autoencoder_transform(result['x'], visible_size, hidden_size, layer_number, patches)[0]

CPU times: user 6.04 s, sys: 772 ms, total: 6.81 s
Wall time: 2min 9s


In [165]:
N, D = activation.shape
n_samples = train['X'].shape[0]
activation = activation.reshape((n_samples, N * D // n_samples))

In [166]:
%time alg_RFC_1.fit(activation, train['y'].reshape(train['y'].shape[0]))

CPU times: user 6.31 s, sys: 48.3 ms, total: 6.36 s
Wall time: 20.4 s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [167]:
%time alg_LR_1.fit(activation, train['y'].reshape(train['y'].shape[0]))

CPU times: user 33.6 s, sys: 141 ms, total: 33.7 s
Wall time: 1min 3s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)

In [168]:
%time patches = image_patches(test['X'], step)

CPU times: user 2.82 s, sys: 2.55 s, total: 5.37 s
Wall time: 3min 32s


In [169]:
%time activation = autoencoder.autoencoder_transform(result['x'], visible_size, hidden_size, layer_number, patches)[0]

CPU times: user 9.47 s, sys: 3.58 s, total: 13 s
Wall time: 5min 7s


In [170]:
N, D = activation.shape
n_samples = test['X'].shape[0]
activation = activation.reshape((n_samples, N * D // n_samples))

In [171]:
%time y_predict_RFC_1 = alg_RFC_1.predict(activation)

CPU times: user 249 ms, sys: 173 ms, total: 423 ms
Wall time: 53.5 s


In [172]:
%time y_predict_LR_1 = alg_LR_1.predict(activation)

CPU times: user 168 ms, sys: 1e+03 ns, total: 168 ms
Wall time: 1.02 s


In [173]:
acc_RFC_1.append(accuracy(test['y'].reshape(test['y'].shape[0]), y_predict_RFC_1))
acc_LR_1.append(accuracy(test['y'].reshape(test['y'].shape[0]), y_predict_LR_1))

<font size=5, face="Times New Roman"> step = 8 </font>

In [194]:
step = 8
%time patches = image_patches(train['X'], step)

CPU times: user 3.36 s, sys: 6.22 s, total: 9.58 s
Wall time: 2min 48s


In [195]:
layer_number = 2
%time activation = autoencoder.autoencoder_transform(result['x'], visible_size, hidden_size, layer_number, patches)[0]

CPU times: user 13.8 s, sys: 6.07 s, total: 19.9 s
Wall time: 11min


In [196]:
N, D = activation.shape
n_samples = train['X'].shape[0]
activation = activation.reshape((n_samples, N * D // n_samples))

In [197]:
%time alg_RFC_1.fit(activation, train['y'].reshape(train['y'].shape[0]))

CPU times: user 9.7 s, sys: 157 ms, total: 9.85 s
Wall time: 23.4 s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [198]:
%time alg_LR_1.fit(activation, train['y'].reshape(train['y'].shape[0]))

CPU times: user 1min 30s, sys: 332 ms, total: 1min 30s
Wall time: 2min


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)

In [199]:
%time patches = image_patches(test['X'], step)

CPU times: user 7.86 s, sys: 14.2 s, total: 22 s
Wall time: 34min 26s


In [200]:
%time activation = autoencoder.autoencoder_transform(result['x'], visible_size, hidden_size, layer_number, patches)[0]

CPU times: user 25.5 s, sys: 21.6 s, total: 47.1 s
Wall time: 52min 24s


In [201]:
N, D = activation.shape
n_samples = test['X'].shape[0]
activation = activation.reshape((n_samples, N * D // n_samples))

In [202]:
%time y_predict_RFC_1 = alg_RFC_1.predict(activation)

CPU times: user 523 ms, sys: 166 ms, total: 689 ms
Wall time: 20.6 s


In [203]:
%time y_predict_LR_1 = alg_LR_1.predict(activation)

CPU times: user 371 ms, sys: 15.7 ms, total: 386 ms
Wall time: 2.34 s


In [204]:
acc_RFC_1.append(accuracy(test['y'].reshape(test['y'].shape[0]), y_predict_RFC_1))
acc_LR_1.append(accuracy(test['y'].reshape(test['y'].shape[0]), y_predict_LR_1))

In [217]:
print(acc_RFC_1)

[0.30462499999999998, 0.33487499999999998, 0.328625, 0.34125, 0.34087499999999998, 0.34712500000000002, 0.35775000000000001]


In [218]:
print(acc_RFC_2)

[0.36299999999999999]


In [219]:
print(acc_LR_1)

[0.33487499999999998, 0.3745, 0.36925000000000002, 0.40412500000000001, 0.41675000000000001, 0.435, 0.46462500000000001]


In [220]:
print(acc_LR_2)

[0.29199999999999998]


<font size=4, face="Times New Roman"> <center> <b> Зависимость точности классификаторов от шага, по которому картинки разбиваются на патчи: </b> </center> </font>

#####Сеть с одним скрытым слоем
|Features (or step)|RandomForest|LogisticRegression|
|------------------|------------|------------------|
|Pixels|0.363|0.292|
|step=32|0.305|0.335|
|step=28|0.335|0.375|
|step=24|0.329|0.369|
|step=20|0.341|0.404|
|step=16|0.34|0.417|
|step=12|0.347|0.435|
|step=8|0.358|0.465|

<font size=4, face="Times New Roman"> Как мы видим, качество логистической регрессии гораздо улучшилось по сравнению с Random Forest. И еще, чем меньше шаг, тем лучше качество классификации, то есть теоретически step=6 качество классификаторов должно также улучшиться, но однако в данной работе не было проведено этого эксперимента в связи с огромной памятью, затрачиваемой для генерации патчей. </font>

##Девятый пункт.

<font size=4, face="Times New Roman"> Необходимо было сравнить сеть с одним скрытым слоем с сетью с тремя скрытыми слоями. </font>

In [8]:
alg_RFC_1 = RFC(n_estimators=20)
alg_LR_1 = LR()

In [16]:
acc_RFC_1 = []
acc_LR_1 = []

In [23]:
hidden_size = np.array([75, 48, 75])
visible_size = 192
theta = autoencoder.initialize(hidden_size, visible_size)
%time patches = sample_patches.sample_patches(unl['X'])
beta = 3
sparsity_param = 1e-2
lambda_ = 1e-5

CPU times: user 15.2 s, sys: 327 ms, total: 15.5 s
Wall time: 54.4 s


In [26]:
result = minimize(J, theta, method='L-BFGS-B', jac=True, tol=1e-7, options={'max_iter': 3000})

In [23]:
with open('train.pk', 'rb') as f:
    train = pickle.load(f)
with open('test.pk', 'rb') as f:
    test = pickle.load(f)

<font size=5, face="Times New Roman"> step = 32 </font>

In [24]:
step = 32
%time patches = image_patches(train['X'], step)

CPU times: user 151 ms, sys: 128 ms, total: 279 ms
Wall time: 392 ms


In [25]:
layer_number = 4
%time activation = autoencoder.autoencoder_transform(result['x'], visible_size, hidden_size, layer_number, patches)[0]

CPU times: user 1.04 s, sys: 230 ms, total: 1.27 s
Wall time: 2.61 s


In [26]:
N, D = activation.shape
n_samples = train['X'].shape[0]
activation = activation.reshape((n_samples, N * D // n_samples))

In [27]:
%time alg_RFC_1.fit(activation, train['y'].reshape(train['y'].shape[0]))

CPU times: user 2.19 s, sys: 0 ns, total: 2.19 s
Wall time: 5.58 s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [28]:
%time alg_LR_1.fit(activation, train['y'].reshape(train['y'].shape[0]))

CPU times: user 2.63 s, sys: 0 ns, total: 2.63 s
Wall time: 3.65 s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)

In [29]:
%time patches = image_patches(test['X'], step)

CPU times: user 261 ms, sys: 305 ms, total: 566 ms
Wall time: 586 ms


In [30]:
%time activation = autoencoder.autoencoder_transform(result['x'], visible_size, hidden_size, layer_number, patches)[0]

CPU times: user 1.54 s, sys: 489 ms, total: 2.03 s
Wall time: 3.66 s


In [31]:
N, D = activation.shape
n_samples = test['X'].shape[0]
activation = activation.reshape((n_samples, N * D // n_samples))

In [32]:
%time y_predict_RFC_1 = alg_RFC_1.predict(activation)

CPU times: user 74.8 ms, sys: 3.22 ms, total: 78 ms
Wall time: 872 ms


In [33]:
%time y_predict_LR_1 = alg_LR_1.predict(activation)

CPU times: user 24.7 ms, sys: 0 ns, total: 24.7 ms
Wall time: 517 ms


In [34]:
acc_RFC_1.append(accuracy(test['y'].reshape(test['y'].shape[0]), y_predict_RFC_1))
acc_LR_1.append(accuracy(test['y'].reshape(test['y'].shape[0]), y_predict_LR_1))

<font size=5, face="Times New Roman"> step = 28 </font>

In [35]:
step = 28
%time patches = image_patches(train['X'], step)

CPU times: user 324 ms, sys: 275 ms, total: 599 ms
Wall time: 660 ms


In [36]:
layer_number = 4
%time activation = autoencoder.autoencoder_transform(result['x'], visible_size, hidden_size, layer_number, patches)[0]

CPU times: user 1.76 s, sys: 404 ms, total: 2.16 s
Wall time: 2.62 s


In [37]:
N, D = activation.shape
n_samples = train['X'].shape[0]
activation = activation.reshape((n_samples, N * D // n_samples))

In [38]:
%time alg_RFC_1.fit(activation, train['y'].reshape(train['y'].shape[0]))

CPU times: user 3.02 s, sys: 0 ns, total: 3.02 s
Wall time: 3.7 s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [39]:
%time alg_LR_1.fit(activation, train['y'].reshape(train['y'].shape[0]))

CPU times: user 4.61 s, sys: 0 ns, total: 4.61 s
Wall time: 4.89 s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)

In [40]:
%time patches = image_patches(test['X'], step)

CPU times: user 463 ms, sys: 585 ms, total: 1.05 s
Wall time: 3.35 s


In [41]:
%time activation = autoencoder.autoencoder_transform(result['x'], visible_size, hidden_size, layer_number, patches)[0]

CPU times: user 2.94 s, sys: 798 ms, total: 3.74 s
Wall time: 10.3 s


In [42]:
N, D = activation.shape
n_samples = test['X'].shape[0]
activation = activation.reshape((n_samples, N * D // n_samples))

In [43]:
%time y_predict_RFC_1 = alg_RFC_1.predict(activation)

CPU times: user 89.2 ms, sys: 3.81 ms, total: 93 ms
Wall time: 420 ms


In [44]:
%time y_predict_LR_1 = alg_LR_1.predict(activation)

CPU times: user 40.6 ms, sys: 0 ns, total: 40.6 ms
Wall time: 113 ms


In [45]:
acc_RFC_1.append(accuracy(test['y'].reshape(test['y'].shape[0]), y_predict_RFC_1))
acc_LR_1.append(accuracy(test['y'].reshape(test['y'].shape[0]), y_predict_LR_1))

<font size=5, face="Times New Roman"> step = 24 </font>

In [46]:
step = 24
%time patches = image_patches(train['X'], step)

CPU times: user 313 ms, sys: 89 ms, total: 402 ms
Wall time: 661 ms


In [47]:
layer_number = 4
%time activation = autoencoder.autoencoder_transform(result['x'], visible_size, hidden_size, layer_number, patches)[0]

CPU times: user 1.79 s, sys: 241 ms, total: 2.03 s
Wall time: 2.03 s


In [48]:
N, D = activation.shape
n_samples = train['X'].shape[0]
activation = activation.reshape((n_samples, N * D // n_samples))

In [49]:
%time alg_RFC_1.fit(activation, train['y'].reshape(train['y'].shape[0]))

CPU times: user 2.96 s, sys: 0 ns, total: 2.96 s
Wall time: 2.94 s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [50]:
%time alg_LR_1.fit(activation, train['y'].reshape(train['y'].shape[0]))

CPU times: user 4.6 s, sys: 0 ns, total: 4.6 s
Wall time: 4.67 s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)

In [51]:
%time patches = image_patches(test['X'], step)

CPU times: user 541 ms, sys: 170 ms, total: 710 ms
Wall time: 726 ms


In [52]:
%time activation = autoencoder.autoencoder_transform(result['x'], visible_size, hidden_size, layer_number, patches)[0]

CPU times: user 2.75 s, sys: 469 ms, total: 3.22 s
Wall time: 3.32 s


In [53]:
N, D = activation.shape
n_samples = test['X'].shape[0]
activation = activation.reshape((n_samples, N * D // n_samples))

In [54]:
%time y_predict_RFC_1 = alg_RFC_1.predict(activation)

CPU times: user 100 ms, sys: 0 ns, total: 100 ms
Wall time: 99.6 ms


In [55]:
%time y_predict_LR_1 = alg_LR_1.predict(activation)

CPU times: user 40.1 ms, sys: 0 ns, total: 40.1 ms
Wall time: 39.8 ms


In [56]:
acc_RFC_1.append(accuracy(test['y'].reshape(test['y'].shape[0]), y_predict_RFC_1))
acc_LR_1.append(accuracy(test['y'].reshape(test['y'].shape[0]), y_predict_LR_1))

<font size=5, face="Times New Roman"> step = 20 </font>

In [57]:
step = 20
%time patches = image_patches(train['X'], step)

CPU times: user 498 ms, sys: 136 ms, total: 634 ms
Wall time: 633 ms


In [58]:
layer_number = 4
%time activation = autoencoder.autoencoder_transform(result['x'], visible_size, hidden_size, layer_number, patches)[0]

CPU times: user 2.7 s, sys: 390 ms, total: 3.09 s
Wall time: 3.09 s


In [59]:
N, D = activation.shape
n_samples = train['X'].shape[0]
activation = activation.reshape((n_samples, N * D // n_samples))

In [60]:
%time alg_RFC_1.fit(activation, train['y'].reshape(train['y'].shape[0]))

CPU times: user 3.83 s, sys: 0 ns, total: 3.83 s
Wall time: 3.82 s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [61]:
%time alg_LR_1.fit(activation, train['y'].reshape(train['y'].shape[0]))

CPU times: user 7.39 s, sys: 0 ns, total: 7.39 s
Wall time: 7.37 s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)

In [62]:
%time patches = image_patches(test['X'], step)

CPU times: user 789 ms, sys: 492 ms, total: 1.28 s
Wall time: 5.14 s


In [63]:
%time activation = autoencoder.autoencoder_transform(result['x'], visible_size, hidden_size, layer_number, patches)[0]

CPU times: user 4.43 s, sys: 1.2 s, total: 5.63 s
Wall time: 11.1 s


In [64]:
N, D = activation.shape
n_samples = test['X'].shape[0]
activation = activation.reshape((n_samples, N * D // n_samples))

In [65]:
%time y_predict_RFC_1 = alg_RFC_1.predict(activation)

CPU times: user 115 ms, sys: 8.35 ms, total: 124 ms
Wall time: 2.4 s


In [66]:
%time y_predict_LR_1 = alg_LR_1.predict(activation)

CPU times: user 63.6 ms, sys: 212 µs, total: 63.8 ms
Wall time: 369 ms


In [67]:
acc_RFC_1.append(accuracy(test['y'].reshape(test['y'].shape[0]), y_predict_RFC_1))
acc_LR_1.append(accuracy(test['y'].reshape(test['y'].shape[0]), y_predict_LR_1))

<font size=5, face="Times New Roman"> step = 16 </font>

In [68]:
step = 16
%time patches = image_patches(train['X'], step)

CPU times: user 715 ms, sys: 904 ms, total: 1.62 s
Wall time: 12 s


In [69]:
layer_number = 4
%time activation = autoencoder.autoencoder_transform(result['x'], visible_size, hidden_size, layer_number, patches)[0]

CPU times: user 4 s, sys: 579 ms, total: 4.58 s
Wall time: 4.6 s


In [70]:
N, D = activation.shape
n_samples = train['X'].shape[0]
activation = activation.reshape((n_samples, N * D // n_samples))

In [71]:
%time alg_RFC_1.fit(activation, train['y'].reshape(train['y'].shape[0]))

CPU times: user 4.55 s, sys: 0 ns, total: 4.55 s
Wall time: 4.81 s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [72]:
%time alg_LR_1.fit(activation, train['y'].reshape(train['y'].shape[0]))

CPU times: user 11 s, sys: 0 ns, total: 11 s
Wall time: 13.7 s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)

In [73]:
%time patches = image_patches(test['X'], step)

CPU times: user 1.15 s, sys: 798 ms, total: 1.95 s
Wall time: 15.7 s


In [74]:
%time activation = autoencoder.autoencoder_transform(result['x'], visible_size, hidden_size, layer_number, patches)[0]

CPU times: user 6.37 s, sys: 2.16 s, total: 8.53 s
Wall time: 27.6 s


In [75]:
N, D = activation.shape
n_samples = test['X'].shape[0]
activation = activation.reshape((n_samples, N * D // n_samples))

In [76]:
%time y_predict_RFC_1 = alg_RFC_1.predict(activation)

CPU times: user 132 ms, sys: 35.6 ms, total: 168 ms
Wall time: 15.5 s


In [77]:
%time y_predict_LR_1 = alg_LR_1.predict(activation)

CPU times: user 93.5 ms, sys: 22 µs, total: 93.5 ms
Wall time: 983 ms


In [78]:
acc_RFC_1.append(accuracy(test['y'].reshape(test['y'].shape[0]), y_predict_RFC_1))
acc_LR_1.append(accuracy(test['y'].reshape(test['y'].shape[0]), y_predict_LR_1))

<font size=5, face="Times New Roman"> step = 12 </font>

In [79]:
step = 12
%time patches = image_patches(train['X'], step)

CPU times: user 1.3 s, sys: 1.13 s, total: 2.43 s
Wall time: 43.8 s


In [80]:
layer_number = 4
%time activation = autoencoder.autoencoder_transform(result['x'], visible_size, hidden_size, layer_number, patches)[0]

CPU times: user 7.23 s, sys: 2.85 s, total: 10.1 s
Wall time: 36.1 s


In [81]:
N, D = activation.shape
n_samples = train['X'].shape[0]
activation = activation.reshape((n_samples, N * D // n_samples))

In [82]:
%time alg_RFC_1.fit(activation, train['y'].reshape(train['y'].shape[0]))

CPU times: user 6.22 s, sys: 15.9 ms, total: 6.24 s
Wall time: 37.7 s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [83]:
%time alg_LR_1.fit(activation, train['y'].reshape(train['y'].shape[0]))

CPU times: user 20.3 s, sys: 93.2 ms, total: 20.4 s
Wall time: 56 s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)

In [84]:
%time patches = image_patches(test['X'], step)

CPU times: user 1.99 s, sys: 3.72 s, total: 5.71 s
Wall time: 1min 54s


In [85]:
%time activation = autoencoder.autoencoder_transform(result['x'], visible_size, hidden_size, layer_number, patches)[0]

CPU times: user 11.7 s, sys: 5.38 s, total: 17 s
Wall time: 1min 58s


In [86]:
N, D = activation.shape
n_samples = test['X'].shape[0]
activation = activation.reshape((n_samples, N * D // n_samples))

In [87]:
%time y_predict_RFC_1 = alg_RFC_1.predict(activation)

CPU times: user 182 ms, sys: 34.7 ms, total: 216 ms
Wall time: 13.9 s


In [88]:
%time y_predict_LR_1 = alg_LR_1.predict(activation)

CPU times: user 166 ms, sys: 0 ns, total: 166 ms
Wall time: 1.15 s


In [89]:
acc_RFC_1.append(accuracy(test['y'].reshape(test['y'].shape[0]), y_predict_RFC_1))
acc_LR_1.append(accuracy(test['y'].reshape(test['y'].shape[0]), y_predict_LR_1))

<font size=5, face="Times New Roman"> step = 8 </font>

In [90]:
step = 8
%time patches = image_patches(train['X'], step)

CPU times: user 2.81 s, sys: 5.8 s, total: 8.61 s
Wall time: 1min 59s


In [91]:
layer_number = 4
%time activation = autoencoder.autoencoder_transform(result['x'], visible_size, hidden_size, layer_number, patches)[0]

CPU times: user 16.1 s, sys: 9.27 s, total: 25.4 s
Wall time: 3min 22s


In [92]:
N, D = activation.shape
n_samples = train['X'].shape[0]
activation = activation.reshape((n_samples, N * D // n_samples))

In [93]:
%time alg_RFC_1.fit(activation, train['y'].reshape(train['y'].shape[0]))

CPU times: user 9.46 s, sys: 160 ms, total: 9.62 s
Wall time: 1min 18s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [94]:
%time alg_LR_1.fit(activation, train['y'].reshape(train['y'].shape[0]))

CPU times: user 50.1 s, sys: 172 ms, total: 50.3 s
Wall time: 1min 31s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)

In [ ]:
%time patches = image_patches(test['X'], step)

CPU times: user 7.03 s, sys: 14.8 s, total: 21.9 s
Wall time: 25min 50s


In [ ]:
%time activation = autoencoder.autoencoder_transform(result['x'], visible_size, hidden_size, layer_number, patches)[0]

CPU times: user 30.8 s, sys: 26.7 s, total: 57.5 s
Wall time: 42min 56s


In [ ]:
N, D = activation.shape
n_samples = test['X'].shape[0]
activation = activation.reshape((n_samples, N * D // n_samples))

In [ ]:
%time y_predict_RFC_1 = alg_RFC_1.predict(activation)

CPU times: user 304 ms, sys: 51 ms, total: 355 ms
Wall time: 4.26 s


In [ ]:
%time y_predict_LR_1 = alg_LR_1.predict(activation)

CPU times: user 386 ms, sys: 2.54 ms, total: 389 ms
Wall time: 1.41 s


In [ ]:
acc_RFC_1.append(accuracy(test['y'].reshape(test['y'].shape[0]), y_predict_RFC_1))
acc_LR_1.append(accuracy(test['y'].reshape(test['y'].shape[0]), y_predict_LR_1))

In [ ]:
print(acc_RFC_1)

[0.29199999999999998, 0.32024999999999998, 0.30175000000000002, 0.32974999999999999, 0.332125, 0.33937499999999998, 0.34637499999999999]


In [ ]:
print(acc_LR_1)

[0.18425, 0.19500000000000001, 0.19112499999999999, 0.21575, 0.22512499999999999, 0.238625, 0.25037500000000001]


<font size=4, face="Times New Roman"> <center> <b> Зависимость точности классификаторов от шага, по которому картинки разбиваются на патчи: </b> </center> </font>

#####Сеть с тремя скрытыми слоями

|Features (or step)|RandomForest|LogisticRegression|
|------------------|------------|------------------|
|Pixels|0.363|0.292|
|step=32|0.292|0.184|
|step=28|0.32|0.195|
|step=24|0.302|0.191|
|step=20|0.33|0.216|
|step=16|0.332|0.225|
|step=12|0.339|0.239|
|step=8|0.346|0.25|

#####Сеть с одним скрытым слоем

|Features (or step)|RandomForest|LogisticRegression|
|------------------|------------|------------------|
|Pixels|0.363|0.292|
|step=32|0.305|0.335|
|step=28|0.335|0.375|
|step=24|0.329|0.369|
|step=20|0.341|0.404|
|step=16|0.34|0.417|
|step=12|0.347|0.435|
|step=8|0.358|0.465|

<font size=4, face="Times New Roman"> Как мы видим сеть с тремя скрытыми слоями работает хуже, чем однослойная, возможно потому, что приходится подбирать больше параметров и проводить кросс-валидацию для поиска оптимальных параметров для сети с тремя скрытыми слоями, но это занимает достаточно много времени. Поэтому предпочтительнее сеть с одним скрытым слоем.</font>

###Десятый пункт.

<font size=4, face="Times New Roman"> В качестве сети была выбрана сеть с одним скрытым слоем, так как показывала более высокую точность и низкое время работы. </font>

In [6]:
with open('unlabeled.pk', 'rb') as f:
    unl = pickle.load(f)

In [7]:
with open('train.pk', 'rb') as f:
    train = pickle.load(f)
with open('test.pk', 'rb') as f:
    test = pickle.load(f)

In [8]:
alg_RFC_1 = RFC(n_estimators=20)
alg_LR_1 = LR()

In [9]:
acc_RFC_1 = []
acc_LR_1 = []

In [10]:
hidden_size = np.array([75])
visible_size = 192
beta = 3
sparsity_param = 1e-2
lambda_ = 1e-5
theta = autoencoder.initialize(hidden_size, visible_size)

<font size=5, face="Times New Roman"> patches = 100 </font>

In [11]:
%time patches = sample_patches.sample_patches(unl['X'], num_patches=100)

CPU times: user 7.94 ms, sys: 0 ns, total: 7.94 ms
Wall time: 1.32 s


In [12]:
%time result = minimize(J, theta, method='L-BFGS-B', jac=True, tol=1e-5, options={'max_iter': 3000})

CPU times: user 12.1 s, sys: 0 ns, total: 12.1 s
Wall time: 12.9 s


/root/anaconda3/lib/python3.4/site-packages/IPython/kernel/__main__.py:1: OptimizeWarning: Unknown solver options: max_iter
  if __name__ == '__main__':


In [13]:
del unl['X']

In [14]:
step = 12
%time patches = image_patches(train['X'], step)

CPU times: user 1.12 s, sys: 1.55 s, total: 2.67 s
Wall time: 3.27 s


In [16]:
layer_number = 2
%time activation = autoencoder.autoencoder_transform(result['x'], visible_size, hidden_size, layer_number, patches)[0]

CPU times: user 5.12 s, sys: 4.59 s, total: 9.7 s
Wall time: 48.8 s


In [17]:
N, D = activation.shape
n_samples = train['X'].shape[0]
activation = activation.reshape((n_samples, N * D // n_samples))

In [18]:
%time alg_RFC_1.fit(activation, train['y'].reshape(train['y'].shape[0]))

CPU times: user 6.23 s, sys: 0 ns, total: 6.23 s
Wall time: 11.2 s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [19]:
%time alg_LR_1.fit(activation, train['y'].reshape(train['y'].shape[0]))

CPU times: user 50.5 s, sys: 0 ns, total: 50.5 s
Wall time: 51 s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)

In [20]:
%time patches = image_patches(test['X'], step)

CPU times: user 2.05 s, sys: 8.55 s, total: 10.6 s
Wall time: 1min 17s


In [21]:
%time activation = autoencoder.autoencoder_transform(result['x'], visible_size, hidden_size, layer_number, patches)[0]

CPU times: user 8.39 s, sys: 8.36 s, total: 16.7 s
Wall time: 2min 2s


In [22]:
N, D = activation.shape
n_samples = test['X'].shape[0]
activation = activation.reshape((n_samples, N * D // n_samples))

In [23]:
%time y_predict_RFC_1 = alg_RFC_1.predict(activation)

CPU times: user 165 ms, sys: 42.4 ms, total: 207 ms
Wall time: 7.47 s


In [24]:
%time y_predict_LR_1 = alg_LR_1.predict(activation)

CPU times: user 169 ms, sys: 1 ms, total: 170 ms
Wall time: 719 ms


In [25]:
acc_RFC_1.append(accuracy(test['y'].reshape(test['y'].shape[0]), y_predict_RFC_1))
acc_LR_1.append(accuracy(test['y'].reshape(test['y'].shape[0]), y_predict_LR_1))

In [26]:
del test['X']
del test['y']
del train['X']
del train['y']

<font size=5, face="Times New Roman"> patches = 1000 </font>

In [27]:
with open('unlabeled.pk', 'rb') as f:
    unl = pickle.load(f)

In [28]:
%time patches = sample_patches.sample_patches(unl['X'], num_patches=1000)

CPU times: user 125 ms, sys: 37.7 ms, total: 163 ms
Wall time: 18.9 s


In [29]:
%time result = minimize(J, theta, method='L-BFGS-B', jac=True, tol=1e-5, options={'max_iter': 3000})

CPU times: user 54.8 s, sys: 0 ns, total: 54.8 s
Wall time: 1min


/root/anaconda3/lib/python3.4/site-packages/IPython/kernel/__main__.py:1: OptimizeWarning: Unknown solver options: max_iter
  if __name__ == '__main__':


In [30]:
del unl['X']

In [31]:
with open('train.pk', 'rb') as f:
    train = pickle.load(f)
with open('test.pk', 'rb') as f:
    test = pickle.load(f)

In [32]:
step = 12
%time patches = image_patches(train['X'], step)

CPU times: user 1.17 s, sys: 5.22 s, total: 6.39 s
Wall time: 6.41 s


In [33]:
layer_number = 2
%time activation = autoencoder.autoencoder_transform(result['x'], visible_size, hidden_size, layer_number, patches)[0]

CPU times: user 4.93 s, sys: 4.32 s, total: 9.25 s
Wall time: 15.2 s


In [34]:
N, D = activation.shape
n_samples = train['X'].shape[0]
activation = activation.reshape((n_samples, N * D // n_samples))

In [35]:
%time alg_RFC_1.fit(activation, train['y'].reshape(train['y'].shape[0]))

CPU times: user 6.29 s, sys: 79.3 ms, total: 6.37 s
Wall time: 15.6 s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [36]:
%time alg_LR_1.fit(activation, train['y'].reshape(train['y'].shape[0]))

CPU times: user 36.3 s, sys: 0 ns, total: 36.3 s
Wall time: 46.9 s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)

In [37]:
%time patches = image_patches(test['X'], step)

CPU times: user 1.87 s, sys: 4.4 s, total: 6.27 s
Wall time: 44.4 s


In [38]:
%time activation = autoencoder.autoencoder_transform(result['x'], visible_size, hidden_size, layer_number, patches)[0]

CPU times: user 8.14 s, sys: 3.58 s, total: 11.7 s
Wall time: 36.6 s


In [39]:
N, D = activation.shape
n_samples = test['X'].shape[0]
activation = activation.reshape((n_samples, N * D // n_samples))

In [40]:
%time y_predict_RFC_1 = alg_RFC_1.predict(activation)

CPU times: user 190 ms, sys: 168 ms, total: 358 ms
Wall time: 30.6 s


In [41]:
%time y_predict_LR_1 = alg_LR_1.predict(activation)

CPU times: user 164 ms, sys: 1.01 ms, total: 165 ms
Wall time: 669 ms


In [42]:
acc_RFC_1.append(accuracy(test['y'].reshape(test['y'].shape[0]), y_predict_RFC_1))
acc_LR_1.append(accuracy(test['y'].reshape(test['y'].shape[0]), y_predict_LR_1))

In [43]:
del test['X']
del test['y']
del train['X']
del train['y']

<font size=5, face="Times New Roman"> patches = 100000 </font>

In [44]:
with open('unlabeled.pk', 'rb') as f:
    unl = pickle.load(f)

In [45]:
%time patches = sample_patches.sample_patches(unl['X'], num_patches=100000)

CPU times: user 26min 9s, sys: 1.03 s, total: 26min 10s
Wall time: 31min 4s


In [46]:
%time result = minimize(J, theta, method='L-BFGS-B', jac=True, tol=1e-5, options={'max_iter': 3000})

CPU times: user 1h 30min 6s, sys: 14min 1s, total: 1h 44min 8s
Wall time: 1h 44min 21s


/root/anaconda3/lib/python3.4/site-packages/IPython/kernel/__main__.py:1: OptimizeWarning: Unknown solver options: max_iter
  if __name__ == '__main__':


In [47]:
del unl['X']

In [48]:
with open('train.pk', 'rb') as f:
    train = pickle.load(f)
with open('test.pk', 'rb') as f:
    test = pickle.load(f)

In [49]:
step = 12
%time patches = image_patches(train['X'], step)

CPU times: user 1.11 s, sys: 968 ms, total: 2.08 s
Wall time: 2.52 s


In [50]:
layer_number = 2
%time activation = autoencoder.autoencoder_transform(result['x'], visible_size, hidden_size, layer_number, patches)[0]

CPU times: user 5.14 s, sys: 1.32 s, total: 6.46 s
Wall time: 13.7 s


In [51]:
N, D = activation.shape
n_samples = train['X'].shape[0]
activation = activation.reshape((n_samples, N * D // n_samples))

In [52]:
%time alg_RFC_1.fit(activation, train['y'].reshape(train['y'].shape[0]))

CPU times: user 6.22 s, sys: 115 ms, total: 6.33 s
Wall time: 26.4 s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [53]:
%time alg_LR_1.fit(activation, train['y'].reshape(train['y'].shape[0]))

CPU times: user 31.4 s, sys: 129 ms, total: 31.5 s
Wall time: 43.1 s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)

In [54]:
%time patches = image_patches(test['X'], step)

CPU times: user 1.9 s, sys: 4.32 s, total: 6.21 s
Wall time: 33.3 s


In [55]:
%time activation = autoencoder.autoencoder_transform(result['x'], visible_size, hidden_size, layer_number, patches)[0]

CPU times: user 8.25 s, sys: 4.64 s, total: 12.9 s
Wall time: 53.2 s


In [56]:
N, D = activation.shape
n_samples = test['X'].shape[0]
activation = activation.reshape((n_samples, N * D // n_samples))

In [57]:
%time y_predict_RFC_1 = alg_RFC_1.predict(activation)

CPU times: user 230 ms, sys: 126 ms, total: 356 ms
Wall time: 10.3 s


In [58]:
%time y_predict_LR_1 = alg_LR_1.predict(activation)

CPU times: user 164 ms, sys: 3.13 ms, total: 168 ms
Wall time: 796 ms


In [59]:
acc_RFC_1.append(accuracy(test['y'].reshape(test['y'].shape[0]), y_predict_RFC_1))
acc_LR_1.append(accuracy(test['y'].reshape(test['y'].shape[0]), y_predict_LR_1))

In [60]:
print(acc_RFC_1)

[0.33624999999999999, 0.34287499999999999, 0.34512500000000002]


In [62]:
print(acc_LR_1)

[0.41087499999999999, 0.43762499999999999, 0.43924999999999997]


<font size=4, face="Times New Roman"> <center> <b> Зависимость точности классификаторов от количества патчей, сгенерированных с unlabeled.pk:</b> </center> </font>

#####step = 12
|Number of patches|RandomForest|LogisticRegression|
|------------------|------------|------------------|
|Pixels|0.363|0.292|
|patches=100|0.336|0.41|
|patches=1000|0.343|0.438|
|patches=10000|0.347|0.435|
|patches=100000|0.345|0.439|

<font size=4, face="Times New Roman"> То есть можно заметить, что при увеличении количества патчей, сгенерированных с unlabeled.pk, увеличивается качество классификации. </font>

In [66]:
del patches